In [43]:
import json
import re
from pathlib import Path
from subprocess import run
from fontTools.ttLib.ttFont import TTFont
from fontTools.pens.ttGlyphPen import TTGlyphPen
from fontTools.pens.boundsPen import BoundsPen
from fontTools.pens.transformPen import TransformPen
from fontTools.misc import transform
from fontTools import svgLib

In [44]:
path_info = Path('svgs/info.json')
path_xml = Path('svgs/default.ttx')
path_save = Path('testfont/result')

In [45]:
path_save.parent.mkdir(exist_ok=True)

In [46]:
with path_info.open() as file:
    data = json.loads(file.read())

info = data[-1]
path = path_info.parent / info['path']
svg = svgLib.path.SVGPath(path)
data = svg.root.find("{http://www.w3.org/2000/svg}g")
tr = data.get('transform')
if tr is not None:
    number = r'([+-]?[0-9]+\.[0-9]+)'
    matchtxt = re.compile(r'translate\('+number+r','+number+r'\) scale\('+number+r','+number+r'\)')
    match = matchtxt.match(tr)
    print(match.groups(), tr)

('0.000000', '352.000000', '0.100000', '-0.100000') translate(0.000000,352.000000) scale(0.100000,-0.100000)


In [47]:
## generate truetype font using fonttools
font = TTFont(sfntVersion="\x00\x01\x00\x00", flavor=False)
font.importXML(path_xml)

with path_info.open() as file:
    data = json.loads(file.read())

sample_text = ''
for info in data:
    name = info['name']
    path = path_info.parent / info['path']
    svg = svgLib.path.SVGPath(path)
    data = svg.root.find("{http://www.w3.org/2000/svg}g")
    if data is not None and data.get('transform', None) is not None:
        tr = data.get('transform', None)
        number = r'([+-]?[0-9]+\.[0-9]+)'
        matchtxt = re.compile(r'translate\('+number+r','+number+r'\) scale\('+number+r','+number+r'\)')
        match = matchtxt.match(tr)
        match = tuple(map(float, match.groups()))
        svg.transform = transform.Identity.translate(match[0], match[1]).scale(match[2], match[3])    
    bpen = BoundsPen(None)
    svg.draw(bpen)
    xMin, yMin, xMax, yMax = bpen.bounds
    scale = min(950/(xMax-xMin), 800/(yMax-yMin))
    dx = (500 - 0.5*scale*(xMin+xMax))
    dy = (500 + 0.5*scale*(yMin+yMax))
    tf = transform.Identity.translate(dx, dy).scale(scale, -scale)
    pen = TTGlyphPen({ord(info['char']): name})
    if svg.transform is None:
        svg.transform = tf
    else:
        svg.transform = tf.transform(svg.transform)
    svg.transform = transform.Identity.translate(0, -200).transform(svg.transform)
    svg.draw(pen)
    glyph = pen.glyph()
    glyph.trim()
    font['glyf'][name] = glyph
    for cmap in font['cmap'].tables:
        cmap.cmap[ord(info['char'])] = name
    sample_text += info['char']
font['name'].addName(sample_text, ((0, 3, 0x0), (1, 0, 0x0), (3, 1, 0x0409)), 18)

for name, glyph in font['glyf'].glyphs.items():
    glyph.recalcBounds(font)
    print(f"{glyph.xMin:6.2f}, {glyph.xMax:6.2f}, {glyph.yMin:6.2f}, {glyph.yMax:6.2f}")
    glyph.xMin = max(0, glyph.xMin)
    glyph.yMin = max(0, glyph.yMin)
    glyph.xMax = min(950, glyph.xMax)
    glyph.yMax = min(950, glyph.yMax)
    if name in font['hmtx'].metrics:
        continue
    font['hmtx'][name] = 950, glyph.xMin

font.setGlyphOrder(font['glyf'].glyphOrder)

for table in font.tables.values():
    if hasattr(table, 'compile'):
        table.compile(font)
    
font.save(path_save.with_suffix('.ttf'))
font.saveXML(path_save.with_suffix('.ttx'))

 33.00, 298.00,   0.00, 666.00
  0.00,   0.00,   0.00,   0.00
  0.00,   0.00,   0.00,   0.00
107.00, 893.00, -100.00, 700.00
101.00, 899.00, -100.00, 700.00
 89.00, 911.00, -100.00, 700.00
100.00, 900.00, -100.00, 700.00
224.00, 777.00, -103.00, 700.00
100.00, 900.00, -100.00, 700.00
 40.00, 960.00, -101.00, 700.00
100.00, 900.00, -100.00, 700.00
 14.00, 979.00,  45.00, 563.00
 92.00, 908.00, -100.00, 702.00
 73.00, 927.00, -100.00, 701.00
217.00, 783.00, -100.00, 700.00
115.00, 884.00, -101.00, 700.00
 85.00, 914.00, -102.00, 700.00
 71.00, 926.00, -110.00, 702.00
151.00, 859.00, -101.00, 700.00
141.00, 860.00, -101.00, 700.00
 20.00, 975.00, -98.00, 695.00
148.00, 851.00, -105.00, 701.00
 25.00, 975.00, -63.00, 660.00
 96.00, 902.00, -100.00, 707.00
229.00, 771.00, -104.00, 701.00


In [48]:
## convert other types based on truetype
try:
    run(['ftcli', 'converter', 'ttf2otf', path_save.with_suffix('.ttf')], capture_output=True)
    (path_save.parent / "result_raw.ttf").unlink(missing_ok=True)
    (path_save.parent / "result_raw.ttx").unlink(missing_ok=True)
    path_save.with_suffix(".ttf").rename(path_save.parent / "result_raw.ttf")
    path_save.with_suffix(".ttx").rename(path_save.parent / "result_raw.ttx")
    run(['ftcli', 'converter', 'otf2ttf', path_save.with_suffix('.otf')], capture_output=True) # better output
    run(['ftcli', 'converter', 'ft2wf', '-f', 'woff', path_save.with_suffix('.ttf')], capture_output=True)
    run(['ftcli', 'converter', 'ft2wf', '-f', 'woff2', path_save.with_suffix('.ttf')], capture_output=True)
    with TTFont(path_save.with_suffix(".ttf")) as font:
        font.saveXML(path_save.parent / "result.ttx")
except FileNotFoundError:
    print("Cannot find font-cli. Please install ftCLI using pip install font-cli")